# Γενετικά δίκτυα

Τα Επαναλαμβανόμενα Νευρωνικά Δίκτυα (RNNs) και οι παραλλαγές τους με πύλες, όπως τα Κύτταρα Μνήμης Μακράς και Βραχείας Διάρκειας (LSTMs) και οι Μονάδες Επαναλαμβανόμενης Πύλης (GRUs), παρέχουν έναν μηχανισμό για τη μοντελοποίηση γλώσσας, δηλαδή μπορούν να μάθουν τη σειρά των λέξεων και να δώσουν προβλέψεις για την επόμενη λέξη σε μια ακολουθία. Αυτό μας επιτρέπει να χρησιμοποιούμε τα RNNs για **γενετικές εργασίες**, όπως η απλή δημιουργία κειμένου, η μηχανική μετάφραση και ακόμη και η περιγραφή εικόνων.

Στην αρχιτεκτονική RNN που συζητήσαμε στην προηγούμενη ενότητα, κάθε μονάδα RNN παρήγαγε την επόμενη κρυφή κατάσταση ως έξοδο. Ωστόσο, μπορούμε επίσης να προσθέσουμε μια άλλη έξοδο σε κάθε επαναλαμβανόμενη μονάδα, η οποία θα μας επιτρέπει να παράγουμε μια **ακολουθία** (που είναι ίση σε μήκος με την αρχική ακολουθία). Επιπλέον, μπορούμε να χρησιμοποιήσουμε μονάδες RNN που δεν δέχονται είσοδο σε κάθε βήμα, αλλά απλώς λαμβάνουν κάποιο αρχικό διάνυσμα κατάστασης και στη συνέχεια παράγουν μια ακολουθία εξόδων.

Σε αυτό το σημειωματάριο, θα επικεντρωθούμε σε απλά γενετικά μοντέλα που μας βοηθούν να δημιουργούμε κείμενο. Για απλότητα, ας κατασκευάσουμε ένα **δίκτυο σε επίπεδο χαρακτήρων**, το οποίο δημιουργεί κείμενο γράμμα προς γράμμα. Κατά τη διάρκεια της εκπαίδευσης, πρέπει να πάρουμε κάποιο σώμα κειμένου και να το χωρίσουμε σε ακολουθίες γραμμάτων.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Δημιουργία λεξιλογίου χαρακτήρων

Για να δημιουργήσουμε ένα γενετικό δίκτυο σε επίπεδο χαρακτήρων, πρέπει να διαχωρίσουμε το κείμενο σε μεμονωμένους χαρακτήρες αντί για λέξεις. Αυτό μπορεί να γίνει ορίζοντας έναν διαφορετικό tokenizer:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Ας δούμε το παράδειγμα του πώς μπορούμε να κωδικοποιήσουμε το κείμενο από το σύνολο δεδομένων μας:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Εκπαίδευση ενός γενετικού RNN

Ο τρόπος με τον οποίο θα εκπαιδεύσουμε το RNN για να δημιουργεί κείμενο είναι ο εξής. Σε κάθε βήμα, θα παίρνουμε μια ακολουθία χαρακτήρων μήκους `nchars` και θα ζητάμε από το δίκτυο να παράγει τον επόμενο χαρακτήρα εξόδου για κάθε χαρακτήρα εισόδου:

![Εικόνα που δείχνει ένα παράδειγμα δημιουργίας της λέξης 'HELLO' από RNN.](../../../../../translated_images/el/rnn-generate.56c54afb52f9781d.webp)

Ανάλογα με το συγκεκριμένο σενάριο, μπορεί να θέλουμε να συμπεριλάβουμε και ειδικούς χαρακτήρες, όπως το *τέλος ακολουθίας* `<eos>`. Στην περίπτωσή μας, θέλουμε απλώς να εκπαιδεύσουμε το δίκτυο για ατελείωτη δημιουργία κειμένου, επομένως θα ορίσουμε το μέγεθος κάθε ακολουθίας να είναι ίσο με `nchars` tokens. Συνεπώς, κάθε παράδειγμα εκπαίδευσης θα αποτελείται από `nchars` εισόδους και `nchars` εξόδους (που είναι η ακολουθία εισόδου μετατοπισμένη κατά ένα σύμβολο προς τα αριστερά). Το minibatch θα αποτελείται από αρκετές τέτοιες ακολουθίες.

Ο τρόπος με τον οποίο θα δημιουργούμε τα minibatches είναι να παίρνουμε κάθε κείμενο ειδήσεων μήκους `l` και να δημιουργούμε όλους τους δυνατούς συνδυασμούς εισόδου-εξόδου από αυτό (θα υπάρχουν `l-nchars` τέτοιοι συνδυασμοί). Αυτοί θα σχηματίζουν ένα minibatch, και το μέγεθος των minibatches θα είναι διαφορετικό σε κάθε βήμα εκπαίδευσης.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Τώρα ας ορίσουμε το δίκτυο του generator. Μπορεί να βασίζεται σε οποιοδήποτε επαναλαμβανόμενο κύτταρο που συζητήσαμε στην προηγούμενη ενότητα (απλό, LSTM ή GRU). Στο παράδειγμά μας θα χρησιμοποιήσουμε LSTM.

Επειδή το δίκτυο λαμβάνει χαρακτήρες ως είσοδο και το μέγεθος του λεξιλογίου είναι αρκετά μικρό, δεν χρειαζόμαστε επίπεδο ενσωμάτωσης (embedding layer), καθώς η είσοδος σε μορφή one-hot μπορεί να περάσει απευθείας στο κύτταρο LSTM. Ωστόσο, επειδή περνάμε αριθμούς χαρακτήρων ως είσοδο, πρέπει να τους μετατρέψουμε σε μορφή one-hot πριν τους περάσουμε στο LSTM. Αυτό γίνεται καλώντας τη συνάρτηση `one_hot` κατά τη διάρκεια της μεθόδου `forward`. Ο κωδικοποιητής εξόδου θα είναι ένα γραμμικό επίπεδο που θα μετατρέπει την κρυφή κατάσταση σε έξοδο σε μορφή one-hot.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

Κατά τη διάρκεια της εκπαίδευσης, θέλουμε να μπορούμε να δειγματοληπτούμε παραγόμενο κείμενο. Για να το κάνουμε αυτό, θα ορίσουμε τη συνάρτηση `generate`, η οποία θα παράγει μια συμβολοσειρά εξόδου μήκους `size`, ξεκινώντας από την αρχική συμβολοσειρά `start`.

Ο τρόπος που λειτουργεί είναι ο εξής. Πρώτα, θα περάσουμε ολόκληρη την αρχική συμβολοσειρά μέσω του δικτύου και θα πάρουμε την κατάσταση εξόδου `s` και τον επόμενο προβλεπόμενο χαρακτήρα `out`. Επειδή το `out` είναι κωδικοποιημένο σε μορφή one-hot, παίρνουμε το `argmax` για να βρούμε τον δείκτη του χαρακτήρα `nc` στο λεξιλόγιο και χρησιμοποιούμε το `itos` για να εντοπίσουμε τον πραγματικό χαρακτήρα και να τον προσθέσουμε στη λίστα χαρακτήρων `chars`. Αυτή η διαδικασία παραγωγής ενός χαρακτήρα επαναλαμβάνεται `size` φορές για να παραχθεί ο απαιτούμενος αριθμός χαρακτήρων.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Ας ξεκινήσουμε την εκπαίδευση! Ο βρόχος εκπαίδευσης είναι σχεδόν ίδιος με όλα τα προηγούμενα παραδείγματα, αλλά αντί για ακρίβεια, εκτυπώνουμε δείγματα παραγόμενου κειμένου κάθε 1000 εποχές.

Ιδιαίτερη προσοχή πρέπει να δοθεί στον τρόπο που υπολογίζουμε την απώλεια. Πρέπει να υπολογίσουμε την απώλεια με δεδομένη την έξοδο που είναι κωδικοποιημένη σε one-hot, `out`, και το αναμενόμενο κείμενο `text_out`, που είναι η λίστα με τους δείκτες χαρακτήρων. Ευτυχώς, η συνάρτηση `cross_entropy` αναμένει ως πρώτο όρισμα την μη κανονικοποιημένη έξοδο του δικτύου και ως δεύτερο τον αριθμό της κλάσης, που είναι ακριβώς αυτό που έχουμε. Επιπλέον, εκτελεί αυτόματα μέσο όρο με βάση το μέγεθος του minibatch.

Περιορίζουμε επίσης την εκπαίδευση σε `samples_to_train` δείγματα, ώστε να μην περιμένουμε υπερβολικά πολύ. Σας ενθαρρύνουμε να πειραματιστείτε και να δοκιμάσετε μεγαλύτερη διάρκεια εκπαίδευσης, πιθανώς για αρκετές εποχές (σε αυτή την περίπτωση θα χρειαστεί να δημιουργήσετε έναν επιπλέον βρόχο γύρω από αυτόν τον κώδικα).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Αυτό το παράδειγμα ήδη παράγει αρκετά καλό κείμενο, αλλά μπορεί να βελτιωθεί περαιτέρω με διάφορους τρόπους:

* **Καλύτερη δημιουργία minibatch**. Ο τρόπος που προετοιμάσαμε τα δεδομένα για εκπαίδευση ήταν να δημιουργούμε ένα minibatch από ένα δείγμα. Αυτό δεν είναι ιδανικό, επειδή τα minibatches έχουν διαφορετικά μεγέθη, και κάποια από αυτά δεν μπορούν καν να δημιουργηθούν, επειδή το κείμενο είναι μικρότερο από `nchars`. Επίσης, τα μικρά minibatches δεν αξιοποιούν επαρκώς την GPU. Θα ήταν πιο σοφό να πάρουμε ένα μεγάλο κομμάτι κειμένου από όλα τα δείγματα, να δημιουργήσουμε όλα τα ζεύγη εισόδου-εξόδου, να τα ανακατέψουμε και να δημιουργήσουμε minibatches ίσου μεγέθους.

* **Πολυεπίπεδο LSTM**. Έχει νόημα να δοκιμάσουμε 2 ή 3 επίπεδα κυττάρων LSTM. Όπως αναφέραμε στην προηγούμενη ενότητα, κάθε επίπεδο του LSTM εξάγει συγκεκριμένα μοτίβα από το κείμενο, και στην περίπτωση ενός γεννήτορα σε επίπεδο χαρακτήρων μπορούμε να περιμένουμε ότι τα χαμηλότερα επίπεδα LSTM θα είναι υπεύθυνα για την εξαγωγή συλλαβών, ενώ τα υψηλότερα επίπεδα - για λέξεις και συνδυασμούς λέξεων. Αυτό μπορεί να υλοποιηθεί απλά περνώντας την παράμετρο αριθμού επιπέδων στον constructor του LSTM.

* Μπορεί επίσης να θέλετε να πειραματιστείτε με **μονάδες GRU** και να δείτε ποιες αποδίδουν καλύτερα, καθώς και με **διαφορετικά μεγέθη κρυφών επιπέδων**. Πολύ μεγάλο κρυφό επίπεδο μπορεί να οδηγήσει σε overfitting (π.χ. το δίκτυο θα μάθει το ακριβές κείμενο), ενώ μικρότερο μέγεθος μπορεί να μην παράγει καλό αποτέλεσμα.


## Γενιά μαλακού κειμένου και θερμοκρασία

Στον προηγούμενο ορισμό της `generate`, πάντα επιλέγαμε τον χαρακτήρα με τη μεγαλύτερη πιθανότητα ως τον επόμενο χαρακτήρα στο παραγόμενο κείμενο. Αυτό είχε ως αποτέλεσμα το κείμενο συχνά να "κυκλώνει" ανάμεσα στις ίδιες ακολουθίες χαρακτήρων ξανά και ξανά, όπως σε αυτό το παράδειγμα:
```
today of the second the company and a second the company ...
```

Ωστόσο, αν κοιτάξουμε την κατανομή πιθανοτήτων για τον επόμενο χαρακτήρα, μπορεί να διαπιστώσουμε ότι η διαφορά ανάμεσα σε μερικές από τις υψηλότερες πιθανότητες δεν είναι μεγάλη, π.χ. ένας χαρακτήρας μπορεί να έχει πιθανότητα 0.2, ενώ ένας άλλος 0.19, κ.λπ. Για παράδειγμα, όταν ψάχνουμε τον επόμενο χαρακτήρα στη σειρά '*play*', ο επόμενος χαρακτήρας μπορεί εξίσου να είναι είτε κενό, είτε **e** (όπως στη λέξη *player*).

Αυτό μας οδηγεί στο συμπέρασμα ότι δεν είναι πάντα "δίκαιο" να επιλέγουμε τον χαρακτήρα με τη μεγαλύτερη πιθανότητα, γιατί η επιλογή του δεύτερου μεγαλύτερου μπορεί επίσης να οδηγήσει σε ουσιαστικό κείμενο. Είναι πιο σοφό να **δειγματοληπτούμε** χαρακτήρες από την κατανομή πιθανοτήτων που δίνεται από την έξοδο του δικτύου.

Αυτή η δειγματοληψία μπορεί να γίνει χρησιμοποιώντας τη συνάρτηση `multinomial`, η οποία υλοποιεί τη λεγόμενη **πολυωνυμική κατανομή**. Μια συνάρτηση που υλοποιεί αυτή τη **μαλακή** γενιά κειμένου ορίζεται παρακάτω:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Έχουμε εισαγάγει μία ακόμη παράμετρο που ονομάζεται **θερμοκρασία**, η οποία χρησιμοποιείται για να υποδείξει πόσο αυστηρά πρέπει να ακολουθούμε την υψηλότερη πιθανότητα. Αν η θερμοκρασία είναι 1.0, κάνουμε δίκαιη δειγματοληψία πολυωνύμου, και όταν η θερμοκρασία τείνει στο άπειρο - όλες οι πιθανότητες γίνονται ίσες, και επιλέγουμε τυχαία τον επόμενο χαρακτήρα. Στο παρακάτω παράδειγμα μπορούμε να παρατηρήσουμε ότι το κείμενο γίνεται χωρίς νόημα όταν αυξάνουμε υπερβολικά τη θερμοκρασία, και μοιάζει με "κυκλικό" κείμενο που παράγεται δύσκολα όταν πλησιάζει το 0.



---

**Αποποίηση Ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν σφάλματα ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
